In [1]:
import numpy as np
import torch
from matplotlib import pyplot as plt

from data_handling.data_loader import load_mavir_data
from trainer_lib import Grid, grid_search
from models import Transformer
import utils

In [2]:
df = load_mavir_data('data/mavir_data/mavir.csv')
df['Power'] = utils.min_max_norm(df['Power'])
sample = utils.sample(df, 10000, start_idx=0)

# imfs, residue = utils.apply_eemd(sample['Power'].to_numpy(), spline_kind='akima')

In [3]:
training_data = np.array(sample['Power'].to_numpy()[...,np.newaxis], dtype=np.float32)
# training_data_decomp = np.array(np.c_[imfs.transpose(), residue[...,np.newaxis]], dtype=np.float32)
# training_data_decomp_rem = np.array(np.c_[imfs[1:].transpose(), residue[...,np.newaxis]], dtype=np.float32)
# training_data_combined = np.array(np.c_[sample['Power'].to_numpy()[...,np.newaxis], imfs.transpose(), residue[...,np.newaxis]], dtype=np.float32)
# training_data_combined_rem = np.array(np.c_[sample['Power'].to_numpy()[...,np.newaxis], imfs[1:].transpose(), residue[...,np.newaxis]], dtype=np.float32)
# print(training_data.shape)
# print(training_data.dtype)
# print(training_data_decomp.shape)
# print(training_data_decomp.dtype)

In [4]:
params = {
    'src_size' : [1*8],
    'tgt_size' : [1*1],
    'd_model' : [256], #, 256, 512],
    'num_heads' : [2], # , 4, 8],
    'num_layers' : [2], # , 2, 3],
    'd_ff' : [512], # , 1024, 2048],
    'enc_seq_length' : [24], # , 96],
    'dec_seq_length' : [1],
    'enc_window': [8],
    'dec_window': [1],
    'dropout' : [0.2], # , 0.1, 0.15, 0.2],
}

grid = Grid(params)
names = utils.generate_name(len(grid), 42)
models = grid_search(grid, names, Transformer, training_data, epochs=50, split=0.1, root_path='./trained/regular/')

Train size: 8806, Validation size: 978
Epoch: 1; Learning rate: [0.0001045]; Train - MSE: 0.4994566842412853; Eval - MSE: 0.20668545480423822, RMSE: 0.45462672029285545, MAE: 0.41446512937545776
Epoch: 2; Learning rate: [0.000204]; Train - MSE: 0.12391752575090215

KeyboardInterrupt: 

In [ ]:

#for model in models_decomp_combined:
#    plt.plot(model['metrics']['eval_loss'], label=f'{model["name"]} - {model["params"]["enc_seq_length"]} - combined')

#for model in models_decomp_combined_rem:
#    plt.plot(model['metrics']['eval_loss'], label=f'{model["name"]} - {model["params"]["enc_seq_length"]} - combined rem')

#for model in models_decomp_rem:
#    plt.plot(model['metrics']['eval_loss'], label=f'{model["name"]} - {model["params"]["enc_seq_length"]} - rem')

#for model in models_decomp:
#    plt.plot(model['metrics']['eval_loss'], label=f'{model["name"]} - {model["params"]["enc_seq_length"]} - decomp')

for model in models:
    plt.plot(model['metrics']['eval']['MSE'], label=f'{model["name"]} - {model["params"]["enc_seq_length"]} - normal - eval')
    plt.plot(model['metrics']['train']['MSE'], label=f'{model["name"]} - {model["params"]["enc_seq_length"]} - normal - train')

plt.legend()
plt.show()

In [ ]:
from trainer_lib.datasets import TimeSeriesWindowedTensorDataset, TimeSeriesWindowedDatasetConfig

shift, look_back, pred = 50, 24, 1

dataset = TimeSeriesWindowedTensorDataset(training_data, TimeSeriesWindowedDatasetConfig((8, 1), (24, 1), 1))
ones = torch.ones(1, 1, dataset[0][1].shape[-1])
for model in models:
    model['model'].eval()
    with torch.no_grad():
    
        ground_truth = []
        predicted = []
        for shift_offset in range(shift, shift+24, 1):
            out = ones
            for i in range(pred):
                out = torch.concatenate((ones, model['model'](dataset[shift_offset][0].unsqueeze(0), out)), dim=1)
        
            predicted.append(dataset.get_sequence_from_y_windows(out[:, 1:, :].detach()))
            ground_truth.append(dataset.get_sequence_from_y_windows(dataset[shift_offset][1]))
    
    predicted = np.array(predicted).reshape(24)
    ground_truth = np.array(ground_truth).reshape(24)
    plt.plot(ground_truth, label='ground truth')
    plt.plot(predicted, label='24h rolling one step')
    
    #output = model['model'](
    #    dataset[shift][0].unsqueeze(0),  
    #    torch.concat((ones, dataset[shift][1][:-1, :].unsqueeze(0)), dim=1)
    #)
    #
    #plt.plot(torch.concat(
    #    (dataset.get_sequence_from_x_windows(dataset[shift][0]), 
    #     dataset.get_sequence_from_y_windows(dataset[shift][1])), dim=0), label='original')
    #plt.plot(
    #    torch.concatenate(
    #        (dataset.get_sequence_from_x_windows(dataset[shift][0]),
    #         dataset.get_sequence_from_y_windows(output[:, :, :].detach())), dim=0), 
    #    label='full access - normal'
    #)
    #plt.plot(
    #    torch.concatenate(
    #        (dataset.get_sequence_from_x_windows(dataset[shift][0]), 
    #         dataset.get_sequence_from_y_windows(out[:, 1:, :].detach())), dim=0),
    #    label='predicted - normal'
    #)
    
#for model in models_decomp:
#    out = torch.ones(1,1,11)
#    for _ in range(25):
#        output = model['model'](torch.tensor(training_data_decomp[np.newaxis, 0:72,:]), out)# torch.tensor(training_data[np.newaxis, 73:97,:]))# torch.zeros((1, 24, 1)))
#        out = torch.concatenate((out, output[:,-1,:].unsqueeze(1)), axis=1)
#    plt.plot(out[:, 1:-1, :].detach().reshape((24,11)).sum(-1), label='predicted - decomp')

#for model in models_decomp_rem:
#    out = torch.ones(1,1,10)
#    for _ in range(25):
#        output = model['model'](torch.tensor(training_data_decomp_rem[np.newaxis, 0:72,:]), out)# torch.tensor(training_data[np.newaxis, 73:97,:]))# torch.zeros((1, 24, 1)))
#        out = torch.concatenate((out, output[:,-1,:].unsqueeze(1)), axis=1)
#    plt.plot(out[:, 1:-1, :].detach().reshape((24,10)).sum(-1), label='predicted - rem')

#for model in models_decomp_combined:
#    out = torch.ones(1,1,1)
#    for _ in range(25):
#        output = model['model'](torch.tensor(training_data_combined[np.newaxis, 0:72,:]), out)# torch.tensor(training_data[np.newaxis, 73:97,:]))# torch.zeros((1, 24, 1)))
#        out = torch.concatenate((out, output[:,-1,:].unsqueeze(1)), axis=1)
#    plt.plot(out[:, 1:-1, :].detach().reshape((24,)), label='predicted - combined')

#for model in models_decomp_combined_rem:
#    out = torch.ones(1,1,1)
#    for _ in range(25):
#        output = model['model'](torch.tensor(training_data_combined_rem[np.newaxis, 0:72,:]), out)# torch.tensor(training_data[np.newaxis, 73:97,:]))# torch.zeros((1, 24, 1)))
#        out = torch.concatenate((out, output[:,-1,:].unsqueeze(1)), axis=1)
#    plt.plot(out[:, 1:-1, :].detach().reshape((24,)), label='predicted - combined rem')

plt.legend()
plt.show()